# Kanon/Honoka classifier using MobileNet_v2

This notebook implements a simple Convolutional Neural Network to classify images of faces of Shibuya Kanon and Kousaka Honoka.

In [ ]:
import re
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfl
import matplotlib as mpl 
import matplotlib.pyplot as plt

In [ ]:
mpl.rcParams['figure.dpi'] = 100

## Loading data

In [ ]:
!tar xf "../input/kanon-honoka-dataset/data.tar.xz"

In [ ]:
# Image paths for 2 classes used in this notebook
KANON_PATH = './Kanon'
HONOKA_PATH = './Honk'

image_size = (192, 192)
batch_size = 32

Prepare a list of files to be loaded

In [ ]:
class_names = ['honoka', 'kanon']
class_idx = {x: i for i, x in enumerate(class_names)}
class_idx

In [ ]:
kanon_list = tf.data.Dataset.list_files(KANON_PATH + '/*.jpg')
honk_list = tf.data.Dataset.list_files(HONOKA_PATH + '/*.jpg')
file_list = kanon_list.concatenate(honk_list)

image_count = file_list.cardinality().numpy()
image_count

In [ ]:
file_list = file_list.shuffle(image_count)

Split the data into train and test set

In [ ]:
test_size = int(image_count * 0.2)
train_ds = file_list.skip(test_size)
test_ds = file_list.take(test_size)

train_ds.cardinality(), test_ds.cardinality()

In [ ]:
val_size = int(test_size * 0.5)
val_ds = test_ds.skip(val_size)
test_ds = test_ds.take(val_size)

val_ds.cardinality(), test_ds.cardinality()

Functions to convert a file path into and `(img, label)` pair

In [ ]:
def get_label(path):
    # Function that takes in a path and return the integral label of the image
    if tf.strings.regex_full_match(path, '.*Kanon.*'):
        return tf.one_hot(class_idx['kanon'], len(class_idx))
    else:
        return tf.one_hot(class_idx['honoka'], len(class_idx))

In [ ]:
def decode_img(img):
    # Decode binary image data into a 3D tensor
    img = tf.io.decode_jpeg(img, channels=3)
    return tf.image.resize(img, image_size, method='area')

In [ ]:
def process_path(path):
    # Function that take in a path and return an (img, label) pair
    label = get_label(path)
    img = tf.io.read_file(path)
    img = decode_img(img)
    return img, label

Apply processing pipeline onto previously read paths

In [ ]:
train_ds = train_ds.map(process_path)
val_ds = val_ds.map(process_path)
test_ds = test_ds.map(process_path)

Split the datasets into batches

In [ ]:
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

## Training the network

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

Define data augmentor layer

In [ ]:
def DataAugmentor():
    model = tf.keras.Sequential()
    model.add(tfl.RandomFlip('horizontal'))
    model.add(tfl.RandomRotation(0.1))
    return model

In [ ]:
augmentor = DataAugmentor()

img = next(iter(train_ds))[0][0]
plt.figure(figsize=(7, 5))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmentor(img) / 255)
    plt.axis('off')

In [ ]:
mobilenet_preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
def HonoKanonClassifier(image_shape=image_size, data_augmentor=DataAugmentor()):
    """
    Function that returns a Model
    
    image_shape (tuple): 2D shape of the image.
    data_augmentor: a Model object which performs data augmentation
    base_freeze_perc: percentage of layers of the base model to freeze
    """
    
    input_shape = image_shape + (3,)
    print(input_shape)
    
    # Create base model to tweak on
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                   include_top=False,  # remove last softmax layer
                                                   weights='imagenet')
    
    base_model.trainable = False
    
    ######### Define model pipeline
    inputs = tfl.Input(shape=input_shape)
    
    x = data_augmentor(inputs)
    x = mobilenet_preprocess_input(x)
    
    # Training=False means do not train during prediction
    x = base_model(x, training=False)
    
    # Similar to Flatten
    x = tfl.GlobalAveragePooling2D()(x)
    # Prevent overfitting
    x = tfl.Dropout(0.2)(x)
    
    # Softmax layer
    outputs = tfl.Dense(2, activation='softmax')(x)
    
    return tf.keras.Model(inputs, outputs)

In [ ]:
model = HonoKanonClassifier()

In [ ]:
model.summary()

In [ ]:
base_learning_rate = 0.001
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
initial_epochs = 10
history = model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=initial_epochs,
    use_multiprocessing=True
)

In [ ]:
def plot_hist(acc, val_acc, loss, val_loss):
    plt.figure(figsize=(5, 7))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training accuracy')
    plt.plot(val_acc, label='Validation accuracy')
    plt.legend()
    
    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training loss')
    plt.plot(val_loss, label='Validation loss')
    plt.legend()
    
    return plt.gcf()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plot_hist(acc, val_acc, loss, val_loss);

## Model fine-tuning

In [ ]:
base_model = model.layers[4]

In [ ]:
# Number of layers in the original base model
n = len(base_model.layers)
n

In [ ]:
freeze_ratio = 0.85 # freeze first 85% of the layers
freeze_layers = int(np.ceil(n * freeze_ratio))

base_model.trainable = False
for layer in base_model.layers[freeze_layers:None]:
    layer.trainable = True

In [ ]:
# Re-compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1 * base_learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
# Re-train the model from previous epoch
fine_history = model.fit(
    train_ds,
    epochs=initial_epochs + 10, # 10 more epochs
    initial_epoch=history.epoch[-1], # continue from last epoch
    validation_data=val_ds
)

In [ ]:
fig = plot_hist(
    acc=acc + fine_history.history['accuracy'],
    val_acc=val_acc + fine_history.history['val_accuracy'],
    loss=loss + fine_history.history['loss'],
    val_loss=val_loss + fine_history.history['val_loss']
)

for ax in fig.get_axes():
    ax.axvline(initial_epochs - 1, color='red', label='Begin fine-tuning')

## Score on test set

In [ ]:
test_data = list(test_ds.as_numpy_iterator())

In [ ]:
x_test, y_test = list(zip(*test_data))

In [ ]:
x_test = np.array(x_test)
y_test = np.array(y_test)

x_test.shape, y_test.shape

In [ ]:
model.evaluate(x_test, y_test)

## Plot several test examples

In [ ]:
random_idx = np.random.randint(0, len(x_test), 10)

x_draw = x_test[random_idx]
y_draw = y_test[random_idx]

In [ ]:
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_draw[i] / 255)
    
    # Size transform
    x_transform = tf.expand_dims(x_draw[i], 0)
               
    pred_vec = model.predict(x_transform)[0]
    pred_class = tf.argmax(pred_vec)
    pred_label = class_names[pred_class]
    pred_prob = "%.02f" % (pred_vec[pred_class] * 100)
    
    plt.title(f'{pred_label} ({pred_prob}%)')
    plt.axis('off')

In [ ]:
model.save('honokanon_model')

## Draw confusion matrix

In [ ]:
def ds_to_numpy(ds, unbatch=True):
    if unbatch:
        ub_ds = ds.unbatch()
    else:
        ub_ds = ds
    ds_lst = list(ub_ds.as_numpy_iterator())
    x, y = list(zip(*ds_lst))
    
    return np.array(x), np.array(y)

Convert datasets to numpy array

In [ ]:
train_np = ds_to_numpy(train_ds)

In [ ]:
test_np = ds_to_numpy(test_ds, unbatch=False)

### Confusion matrix for training data

In [ ]:
train_x, train_y = train_np
test_x, test_y = test_np

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

Get training predictions

In [ ]:
train_pred = model.predict(train_x)

In [ ]:
train_pred_classes = np.apply_along_axis(lambda x: np.argmax(x), 1, train_pred)

In [ ]:
train_classes = np.apply_along_axis(lambda x: np.argmax(x), 1, train_y)

Plot confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
def plot_confusion_matrix(cm):
    sns.heatmap(cm, annot=True, fmt='g')

    ax = plt.gca()
    xticklabels = ax.get_xticklabels()
    new_xlabels = list(map(lambda x: class_names[x], map(lambda x: int(x.get_text()), xticklabels)))
    ax.set_xticklabels(new_xlabels)

    yticklabels = ax.get_yticklabels()
    new_ylabels = list(map(lambda x: class_names[x], map(lambda x: int(x.get_text()), yticklabels)))
    ax.set_yticklabels(new_ylabels)

In [ ]:
cm = confusion_matrix(train_classes, train_pred_classes)
plot_confusion_matrix(cm)

### Confusion matrix for test data

In [ ]:
test_pred = model.predict(test_x)

In [ ]:
test_pred_classes = np.apply_along_axis(lambda x: np.argmax(x), 1, test_pred)

In [ ]:
test_classes = np.apply_along_axis(lambda x: np.argmax(x), 1, test_y)

In [ ]:
cm = confusion_matrix(test_classes, test_pred_classes)
plot_confusion_matrix(cm)